# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [97]:
import pandas as pd
data = pd.read_csv("datasets/data.csv")
print(data.head(10))

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

Были обнаружены пропуски в столбацах: total_income, days_employed. 

In [98]:
col_nan_total_income = data["total_income"].isna()
print(len(col_nan_total_income)/21524)

1.0000464597658427


Процент пропусков составили чуть более 1%. Вероятно пропуски появились из-за отсутствия основного места работы у клиента, следовательно не были внесены из официальных баз. Лучшее решение заменить пропуски на медианное значение, так как среднее будет не характерно описывать картну в общем.

### Шаг 2.1 Заполнение пропусков

In [99]:

median_of_total_income = data["total_income"].median() #Медианное значение по столбцу - total_income
data["total_income"] = data["total_income"].fillna(value=median_of_total_income) #Замена пропусков на медианное значение в столбце - total_income
print(data.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [100]:
anom = data[data["days_employed"] < 0 ]["days_employed"].count()
print(anom) # 15906 аномалей было обнаружено в столбце - "days_employed"
print(data[data["days_employed"] == 0 ]["days_employed"].count()) #Количество строк с 0 значением

15906
0


Таким образом, прежде чем исправлять аномалию, нужно все пропуски заменить на нулевое значение, затем исправить аномалию, а потом заменить нули на медианное значение по столбцу.

In [101]:
#data["days_employed"]=data["days_employed"].fillna(value=0) #Заменяем пустоту нулями.
def change(df):
    df["days_employed"] = df["days_employed"].apply(lambda x: abs(x) if pd.notna(x) and x<0 else x)
    return df
data = change(data)

median_of_days_employed = data["days_employed"].median() #Медианное значение по столбцу - days_employed
data["days_employed"] = data["days_employed"].fillna(value=median_of_days_employed) #Замена пропусков на медианное значение в столбце - days_employed.
print(data.head(15)) # Обновлённый датасет 

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42               высшее             0   
1          1    4024.803754         36              среднее             1   
2          0    5623.422610         33              Среднее             1   
3          3    4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0     926.185831         27               высшее             0   
6          0    2879.202052         43               высшее             0   
7          0     152.779569         50              СРЕДНЕЕ             1   
8          2    6929.865299         35               ВЫСШЕЕ             0   
9          0    2188.756445         41              среднее             1   
10         2    4171.483647         36               высшее             0   
11         0     792.701887         40              среднее             1   

### Шаг 2.3. Изменение типов данных.

In [102]:
data["total_income"] = data["total_income"].astype('int')
print(data.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42               высшее             0   
1          1    4024.803754         36              среднее             1   
2          0    5623.422610         33              Среднее             1   
3          3    4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0     926.185831         27               высшее             0   
6          0    2879.202052         43               высшее             0   
7          0     152.779569         50              СРЕДНЕЕ             1   
8          2    6929.865299         35               ВЫСШЕЕ             0   
9          0    2188.756445         41              среднее             1   
10         2    4171.483647         36               высшее             0   
11         0     792.701887         40              среднее             1   

### Шаг 2.4. Удаление дубликатов.

In [103]:
data["education"] = data['education'].str.lower()
data = data.drop_duplicates().reset_index(drop=True)
print(data.head(15))

    children  days_employed  dob_years            education  education_id  \
0          1    8437.673028         42               высшее             0   
1          1    4024.803754         36              среднее             1   
2          0    5623.422610         33              среднее             1   
3          3    4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0     926.185831         27               высшее             0   
6          0    2879.202052         43               высшее             0   
7          0     152.779569         50              среднее             1   
8          2    6929.865299         35               высшее             0   
9          0    2188.756445         41              среднее             1   
10         2    4171.483647         36               высшее             0   
11         0     792.701887         40              среднее             1   

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [104]:
education_df = data[["education", "education_id"]].drop_duplicates().reset_index(drop=True)
family_status_df = data[["family_status", "family_status_id"]].drop_duplicates().reset_index(drop=True)
print(education_df)
print(family_status_df)
data = data.drop(["education", "family_status"], axis=1)
#print(data.head(10))

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4
           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4


### Шаг 2.6. Категоризация дохода.

In [105]:
def income_category(row):
    if(row["total_income"] <= 30000):
        return "E"
    elif(row["total_income"] <= 50000):
        return "D"
    elif(row["total_income"] <= 200000):
        return "C"
    elif(row["total_income"] <= 1000000):
        return "B"
    else:
        return "A"
data["total_income_category"] = data.apply(income_category, axis=1)
print(data.head(15))

    children  days_employed  dob_years  education_id  family_status_id gender  \
0          1    8437.673028         42             0                 0      F   
1          1    4024.803754         36             1                 0      F   
2          0    5623.422610         33             1                 0      M   
3          3    4124.747207         32             1                 0      M   
4          0  340266.072047         53             1                 1      F   
5          0     926.185831         27             0                 1      M   
6          0    2879.202052         43             0                 0      F   
7          0     152.779569         50             1                 0      M   
8          2    6929.865299         35             0                 1      F   
9          0    2188.756445         41             1                 0      M   
10         2    4171.483647         36             0                 0      M   
11         0     792.701887 

### Шаг 2.7. Категоризация целей кредита.

In [108]:
#print(data["purpose"].unique())
def purpose_category(row):
    if ("образ" in row["purpose"]):
        return "Получение образования"
    elif ("авто" in row["purpose"]):
        return "Операции с автомобилем"
    elif(("жиль" in row["purpose"]) or ("недвиж" in row["purpose"])):
        return "Операции с недвижимостью"
    else:
        return "Проведение свадьбы"
data["purpose_category"] = data.apply(purpose_category, axis=1)
print(data.head(15))

    children  days_employed  dob_years  education_id  family_status_id gender  \
0          1    8437.673028         42             0                 0      F   
1          1    4024.803754         36             1                 0      F   
2          0    5623.422610         33             1                 0      M   
3          3    4124.747207         32             1                 0      M   
4          0  340266.072047         53             1                 1      F   
5          0     926.185831         27             0                 1      M   
6          0    2879.202052         43             0                 0      F   
7          0     152.779569         50             1                 0      M   
8          2    6929.865299         35             0                 1      F   
9          0    2188.756445         41             1                 0      M   
10         2    4171.483647         36             0                 0      M   
11         0     792.701887 

### Ответы на вопросы.

##### Вопрос 1:

##### Вывод 1:

## Общий вывод: